# Phase I stage for the analytic center computation

# The set-up

In [7]:
import numpy as np
import scipy.optimize as opt

$\DeclareMathOperator{\domain}{dom}
\newcommand{\transpose}{\text{T}}
\newcommand{\vec}[1]{\begin{pmatrix}#1\end{pmatrix}}$

# Theory and implementation

The analytic center is the solution of the problem
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=1}^{m}{\log{(b_i - a_i^\transpose x)}}
\end{equation*}
where 
\begin{equation*}
    \domain \phi = \{x \;|\; a_i^\transpose x < b_i, i = 1, \dots, m\}.
\end{equation*}
Our approach in solving this is to
  1. use a phase I optimizatin method to find a point in $\domain \phi$; and then
  2. use this as the initial point for Newton's method to solve our problem. 
  
Here we focus on 1. Now, the phase I optimization problem for this problem's inequalities is 
\begin{align*}
    &\text{minimize} \quad s  \\
    &\text{subject to} \quad a_i^\transpose x - b_i \leq s, i = 1, \dots, m. \\
\end{align*}
This is always strictly feasible as we can pick any $x^{(0)}$ in the domain of all $f_i$ and then any initial $s > \max_{i=1, \dots, m}\{a_i^\transpose x^{(0)} - b_i\}$. We re-write the problem as
\begin{align*}
    &\text{minimize} \quad s  \\
    &\text{subject to} \quad a_i^\transpose x - b_i - s \leq 0, i = 1, \dots, m, \\
\end{align*}
where $z = (x, s) \in \mathbb{R}^n \times \mathbb{R}$. Writing the inequality constraints implicity, equivalently we have
\begin{align*}
    \text{minimize} \quad s + \sum_{i=1}^{m}{I_{-}{(a_i^\transpose x - b_i - s)}}. 
\end{align*}
As we can approximate $I_{-}$ using the log barrier function we therefore have the approximate problem
\begin{align*}
    \text{minimize} \quad s + \sum_{i=1}^{m}-\left(\frac{1}{t}\right)\log{(-(a_i^\transpose x - b_i - s))},
\end{align*}
or equivalently
\begin{align*}
    \text{minimize} \quad f_0(z) = ts + \sum_{i=1}^{m}-\log{(s + b_i - a_i^\transpose x)}.
\end{align*}

For the gradient we have 
\begin{align*}
    &[\nabla f_0(z)]_{i=1}^n = \sum_{i=1}^{m} \frac{1}{s+b_i-a_i^\transpose x}a_i = A^{\transpose} d  \text{ where } d = [\frac{1}{s + b_i - a_i^\transpose x}]_{i=1}^m, \\ 
    &[\nabla f_0(z)]_{n+1} = t - \sum_{i=1}^{m} \frac{1}{s+b_i-a_i^\transpose x}. 
\end{align*}

For the Hessian we have
\begin{align*}
    &[H f_0(z)]_{(i, j), 1 \leq i, j \leq n} = 
    \sum_{i=1}^{m} \frac{1}{(s+b_i-a_i^\transpose x)^2}a_i a_i^\transpose = 
    A^{\transpose} \text{diag}(d)^2 A \\ 
    &[H f_0(z)]_{(i, n+1), 1 \leq i \leq n} = 
    -\sum_{i=1}^{m} \frac{1}{(s+b_i-a_i^\transpose x)^2}a_i =
    A^{\transpose} [\frac{-1}{(s+b_i-a_i^\transpose x)^2}]_{i=1}^m\\
    &[H f_0(z)]_{(n+1, j), 1 \leq j \leq n} = 
    [H f_0(z)]_{(i, n+1), 1 \leq i \leq n}^{\transpose} \\
    &[H f_0(z)]_{(n+1, n+1)} = \sum_{i=1}^{m} \frac{1}{(s+b_i-a_i^\transpose x)^2}.
\end{align*}

Following this, the implementation of the objective function, its gradient and Hessian are as follows:

In [8]:
def phase_i_func(z, t, A, b):
    """
    Returns the value at z of the objective function of the phase I 
    optimization problem which has been approximated using the log barrier 
    function. 
    """
    x = z[:-1]
    x = np.reshape(x, (x.shape[0], 1))
    s = z[-1]
    # return t*s - np.log(np.prod(s + (b-np.dot(A, x))))
    return t*s - np.sum(np.log(s + (b-np.dot(A, x))))

def phase_i_grad(z, t, A, b):
    """
    Returns the gradient of the objective function at z of the phase I 
    optimization problem which has been approximated using the log barrier 
    function. 
    """
    x = z[:-1]
    s = z[-1]
    d = 1./(s + (b-np.dot(A, x)))
    grad_x = np.dot(np.transpose(A), d)
    grad_s = t - np.sum(d)
    grad = np.append(grad_x, grad_s)
    return grad

def phase_i_hess(z, t, A, b):
    """
    Returns the Hessian of the objective function at z of the phase I 
    optimization problem which has been approximated using the log barrier 
    function. 
    """
    x = z[:-1]
    s = z[-1] 
    d = 1./(s + (b-np.dot(A,x)))
    diagd = np.diag(d)
    d_sqd = np.power(d, 2)
    hess_xx = np.dot(np.dot(np.transpose(A), np.linalg.matrix_power(diagd, 2))
                     , A)
    hess_xs = -1 * np.dot(np.transpose(A), d_sqd)
    hess_sx = np.append(hess_xs, np.sum(d_sqd))
    hess_xs = np.reshape(hess_xs, (hess_xs.shape[0],1))
    hess = np.hstack((hess_xx, hess_xs))
    hess = np.vstack((hess, hess_sx))
    return hess

We can then implement the barrier method as follows: 

# The example

The example we consider is to find a strictly feasible point in $\mathbb{R}^2$ of the inequalities $x_1 \leq 1, x_1 \geq 0, x_2 \leq 1, x_2 \geq 0$. This is a unit square centered at $(\frac{1}{2}, \frac{1}{2})$ and as we expect $x_{ac} = (\frac{1}{2}, \frac{1}{2})$. Following the notation above we have 
\begin{align*}
    &a_1 = \begin{bmatrix}1\\0\end{bmatrix}, &&b_1 = 1, \\
    &a_2 = \begin{bmatrix}-1\\0\end{bmatrix}, &&b_2 = 0, \\
    &a_3 = \begin{bmatrix}0\\1\end{bmatrix}, &&b_3 = 1, \\
    &a_4 = \begin{bmatrix}0\\-1\end{bmatrix}, &&b_4 = 0. 
\end{align*}

We test $\texttt{phase_i_func, phase_i_grad}$ and $\texttt{phase_i_hess}$: 

A toy computation:

In [16]:
A=np.array([[1, 0],[-1,0],[0,1],[0,-1]])
b=np.array([1, 0, 1, 0])
x = np.zeros(A[0].shape[0])
s = 1 + np.amax(-b)
z = np.append(x, s)
t = 10

opt.minimize(phase_i_func, z, args = (t, A, b),
             method='Newton-CG', jac=phase_i_grad, hess = phase_i_hess)

/Users/davidjwu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in log
/Users/davidjwu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in log
/Users/davidjwu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in log
/Users/davidjwu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in log
/Users/davidjwu/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in log


     fun: nan
     jac: array([  1.59917696e-10,   1.59917696e-10,   1.00000000e+01])
 message: 'Optimization terminated successfully.'
    nfev: 84
    nhev: 3
     nit: 3
    njev: 84
  status: 0
 success: True
       x: array([  8.28448507e+10,   8.28448507e+10,  -8.88783601e+10])

The implementation of the phase I optimization procedure is then

In [10]:
def phase_i_opt(A, b, t=4, mu=15, maxiter=2500):
    """
    Returns a point x that satisfies Ax<b, computed using the barrier method.
    """
    x = np.zeros((A[0].shape[0],1))
    s = 0.1*np.fabs(np.amax(-b)) + np.amax(-b)
    z = np.vstack((x, s))
    i = 0
    while z[-1] >= 0 and 4/t >= 1e-5:
        result = opt.minimize(phase_i_func, z, args = (t, A, b),
                              method='Newton-CG', 
                              jac=phase_i_grad, hess = phase_i_hess)
        z = result.x
        t = mu*t
        i = i + 1
    return z